# Project 1: Trading with Momentum
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it Udacity.
## Packages
When you implement the functions, you'll only need to use the [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/) packages. Don't import any other packages, otherwise the grader willn't be able to run your code.

The other packages that we're importing is `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
The data source we'll be using is the [Wiki End of Day data](https://www.quandl.com/databases/WIKIP) hosted at [Quandl](https://www.quandl.com). This contains data for many stocks, but we'll just be looking at the S&P 500 stocks. We'll also make things a little easier to solve by narrowing our range of time from 2007-06-30 to 2017-09-30.
### Set API Key
Set the `quandl_api_key` variable to your Quandl api key. You can find your Quandl api key [here](https://www.quandl.com/account/api).

In [ ]:
# TODO: Add your Quandl API Key
quandl_api_key  = ''

### Download Data

In [ ]:
import os

snp500_file_path = 'data/tickers_SnP500.txt'
wiki_file_path = 'data/WIKI_PRICES.csv'
start_date, end_date = '2013-07-01', '2017-06-30'
use_columns = ['date', 'ticker', 'adj_open', 'adj_close', 'adj_high', 'adj_low']

if not os.path.exists(wiki_file_path):
    with open(snp500_file_path) as f:
        tickers = f.read().split()
    
    helper.download_quandl_dataset(quandl_api_key, 'WIKI', 'PRICES', wiki_file_path, use_columns, tickers, start_date, end_date)
else:
    print('Data already downloaded')

### Load Data

In [ ]:
df = pd.read_csv(wiki_file_path, parse_dates=['date'], index_col=False)

print('Loaded Dataframe')

### View Data
In this project, you won't be building any charts.  We will provide all the code to plot or graph the data using our `project_helper` package. These charts will help you understand the data that you're working with.

Let's see what a single stock looks like from the DataFrame we loaded, called `df`. For this example and future display examples, we'll use Apple's stock "AAPL". Run the code below to view a candlestick chart of Apple stock.

In [ ]:
apple_ticker = 'AAPL'
project_helper.plot_stock(df[df['ticker'] == apple_ticker], '{} Stock'.format(apple_ticker))

### 2-D Matrices
Here we convert `df` into multiple DataFrames for each OHLC. We could use a multiindex, but that just stacks the columns for each ticker. We want to be able to apply calculations without using `groupby` each time.

In [ ]:
adj_open = df.reset_index().pivot(index='date', columns='ticker', values='adj_open')
adj_high = df.reset_index().pivot(index='date', columns='ticker', values='adj_high')
adj_low = df.reset_index().pivot(index='date', columns='ticker', values='adj_low')
adj_close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')

### View Data
Run the cell below to see what the data looks like for `adj_close`.

In [ ]:
project_helper.print_dataframe(adj_close)

## Resample Adjusted Prices

The trading signal you'll develop in this project does not need to be based on daily prices, for instance, you can use month-end prices to perform trading once a month. To do this, you must first resample the daily adjusted closing prices into monthly buckets, and select the last observation of each month.

Implement the `resample_prices` to resample `df` at the sampling frequency of `freq`.

In [ ]:
def resample_prices(prices, freq='M'):
    """
    Resample prices for each ticker at specified frequency.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_resample_prices(resample_prices)

### View Data
Let's apply this function to `adj_close` and view the results.

In [ ]:
monthly_close = resample_prices(adj_close)
project_helper.plot_resampled_prices(
    monthly_close.loc[:, apple_ticker],
    adj_close.loc[:, apple_ticker],
    '{} Stock - Close Vs Monthly Close'.format(apple_ticker))

## Compute Momentum Indicator

Compute log returns ($r_t$) from prices ($x_t$) as your primary momentum indicator:

$$r_t = log_e(x_t) - log_e(x_{t-1})$$

Implement the `compute_log_returns` function below, such that it accepts a dataframe (like one returned by `resample_prices`), and produces a similar dataframe of log returns.

In [ ]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_compute_log_returns(compute_log_returns)

### View Data
Using the same data returned from `resample_prices`, we'll generate the log returns.

In [ ]:
monthly_close_returns = compute_log_returns(monthly_close)
project_helper.plot_returns(
    monthly_close_returns.loc[:, apple_ticker],
    'Log Returns of {} Stock (Monthly)'.format(apple_ticker))

## Shift Returns
Implement the `shift_returns` function to shift the log returns to the previous or future returns in the time series. For example, the parameter `shift_n` is 2 and `df` is the following:

```
        2013-07-08  2013-07-09  2013-07-10  2013-07-11  ...
A       0.015       0.082       0.096       0.020       ...
B       0.037       0.095       0.027       0.063       ...
C       0.094       0.001       0.093       0.019       ...
D       0.092       0.057       0.069       0.087       ...
...     ...         ...         ...         ...
```

the output of the `shift_returns` function would be:

```
        2013-07-08  2013-07-09  2013-07-10  2013-07-11  ...
A       NaN         NaN         0.015       0.082       ...
B       NaN         NaN         0.037       0.095       ...
C       NaN         NaN         0.094       0.001       ...
D       NaN         NaN         0.092       0.057       ...
...     ...         ...         ...         ...
```
Using the same `df` as above, the `shift_returns` function should generate the following with `shift_n` as -2:
```
        2013-07-08  2013-07-09  2013-07-10  2013-07-11  ...
A       0.096       0.020       ...         ...         ...
B       0.027       0.063       ...         ...         ...
C       0.093       0.019       ...         ...         ...
D       0.069       0.087       ...         ...         ...
...     ...         ...         ...         ...
```
_Note: The "..." represents data points we're not showing._

In [ ]:
def shift_returns(returns, shift_n):
    """
    Generate shifted returns
    
    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    shift_n : int
        Number of periods to move, can be positive or negative
    
    Returns
    -------
    shifted_returns : DataFrame
        Shifted returns for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_shift_returns(shift_returns)

### View Data
Let's get the previous month's and next month's returns.

In [ ]:
prev_returns = shift_returns(monthly_close_returns, 1)
lookahead_returns = shift_returns(monthly_close_returns, -1)

project_helper.plot_shifted_returns(
    prev_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Previous Returns of {} Stock'.format(apple_ticker))
project_helper.plot_shifted_returns(
    lookahead_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Lookahead Returns of {} Stock'.format(apple_ticker))

## Generate Trading Signal

A trading signal is a sequence of trading actions, or results that can be used to take trading actions. A common form is to produce a "long" and "short" portfolio of stocks on each date (e.g. end of each month, or whatever frequency you desire to trade at). This signal can be interpreted as rebalancing your portfolio on each of those dates, entering long ("buy") and short ("sell") positions as indicated.

Here's a strategy that we will try:
> For each month-end observation period, rank the stocks by _previous_ returns, from the highest to the lowest. Select the top performing stocks for the long portfolio, and the bottom performing stocks for the short portfolio.

Implement the `get_top_n` function to get the top performing stock for each month. Get the top performing stocks from `prev_returns` by assigning them a value of 1. For all other stocks, give them a value of 0. For example, using the following `prev_returns`:
```
        2013-07-08  2013-07-09  ...
A       0.015       0.082       ...
B       0.037       0.095       ...
C       0.094       0.001       ...
D       0.092       0.057       ...
E       0.072       0.099       ...
F       0.083       0.038       ...
G       0.090       0.045       ...
```
The function `get_top_n` with `top_n` set to 3 should return the following:
```
        2013-07-08  2013-07-09  ...
A       0           1           ...
B       0           1           ...
C       1           0           ...
D       1           0           ...
E       0           1           ...
F       0           0           ...
G       1           0           ...
```
Note: You can use Pandas's nlargest function to help you implement `get_top_n` with the `keep` parameter equal to "first".

In [ ]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    # TODO: Implement Function
    
    return None

project_tests.test_get_top_n(get_top_n)

### View Data
We want to get the best performing and worst performing stocks. To get the best performing stocks, we'll use the `get_top_n` function. To get the worst performing stocks, we'll also use the `get_top_n` function. However, we pass in `-1*prev_returns` instead of just `prev_returns`. Multiply by negative one will flip all the positive returns to negative and negative returns to positive. Thus, it returns the top worst performing stocks.

In [ ]:
top_bottom_n = 50
df_long = get_top_n(prev_returns, top_bottom_n)
df_short = get_top_n(-1*prev_returns, top_bottom_n)
project_helper.print_top(df_long, 'Longed Stocks')
project_helper.print_top(df_short, 'Shorted Stocks')

## Projected returns
It's now time to check if your trading signal has the potential to become profitable!

We'll start by computing the net returns this portfolio would return. For simplicity, we'll assume every stock gets an equal dollar amount of investment. This makes it easier to compute a portfolio's returns as the simple arithmetic average of the individual stock returns.

Implement the `portfolio_returns` function to compute the expected portfolio returns. Using `df_long` to indicate which stocks to long and `df_short` to indicate which stocks to short, calculate the returns using `lookahead_returns`. To help with calculation, we've provided you with `n_stocks` as the number of stocks we're investing in a single period.

In [ ]:
def portfolio_returns(df_long, df_short, lookahead_returns, n_stocks):
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """
    # TODO: Implement Function
    
    return None

project_tests.test_portfolio_returns(portfolio_returns)

### View Data
Time to see how the portfolio did.

In [ ]:
expected_portfolio_returns = portfolio_returns(df_long, df_short, lookahead_returns, 2*top_bottom_n)
project_helper.plot_returns(expected_portfolio_returns.T.sum(), 'Portfolio Returns')

Now that you have your net returns, compute their mean to see if it is positive.

In [ ]:
# TODO: Compute mean of net returns
mu = 0.0
print("mean return = {:.6f}".format(mu))
# Note: This is mean monthly *log* return if your trading frequency is once a month

# TODO(optional): Compute annualized rate of return
mu_arr = 0.0
print("annualized rate of return = {:.2f}%".format(mu_arr * 100))

Note that this may be a very small number, since it is the mean _log return_ you would get over your chosen trading period (e.g. a _month_). To convert it into a more intuitive measure, you may compute an annualized percentage rate of return (_hint: simply multiplying monthly returns by 12 would be incorrect_). This is just to give you a sense of what the log returns are indicating; it would be naive to assume that rate of return to hold over any significant period of time.

If you get a negative number here, that means following your trading signal would've most likely resulted in losses during the period tested. Try to go back and change the parameters, or your logic for selecting stocks, so that you get positive mean returns. Please keep the structure of the signal the same - it should be based on a momentum indicator like log returns, and should produce a long/short portfolio.

### Statistical test

Even if you get positive returns, it may be a result of the sample of data points chosen, and not a good predictor of your signal's general performance. To try and rule out this possibility, we'll perform a statistical test on the results.

Our null hypothesis ($H_0$) is that the expected mean return from the signal is zero, and that any positive mean observed here is a matter of chance (within certain expected bounds of deviation). We'll perform a one-sample, one-sided t-test on the observed net returns, to see if we can reject $H_0$.

We'll need to first compute the t-statistic, and then find its corresponding p-value. The p-value will indicate the probability of observing these net returns if the null hypothesis were true. Therefore, a smaller p-value would indicate that the null hypothesis is less likely. In fact, it's good practice to set a desired level of significance or alpha ($\alpha$) _before_ computing the p-value, and then reject the null hypothesis if $p < \alpha$.

Let's choose $\alpha = 0.10$, i.e. we'll be happy with a 10% risk that the signal is actually junk, or that we want to be at least 90% confident that the signal produces positive net returns.

In [ ]:
from scipy import stats

def analyze_alpha(expected_portfolio_returns):
    """
    Analyze returns to check if mean is significantly above zero.
    
    Compute the mean and standard error, then perform a t-test,
    with the null hypothesis being that the expected mean return is zero.
    Return the resulting t-statistic and p-value (significance).
    
    Parameters
    ----------
    expected_portfolio_returns : Pandas Series
        Expected portfolio returns for each ticker and date
    
    Returns
    -------
    mean
        Mean of net portfolio returns
    ste
        Standard error of the mean
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    # TODO: Compute mean, standard error

    # TODO: Perform t-test

    return None

project_tests.test_analyze_alpha(analyze_alpha)

### View Data

In [ ]:
mean, ste, t_value, p_value = analyze_alpha(expected_portfolio_returns.sum().dropna())
print("""
Alpha analysis:
 mean:           {:.6f}
 standard error: {:.6f}
 t-value:        {:.3f}
 p-value:        {:.6f}
""".format(mean, ste, t_value, p_value))

---

**Q**: What p-value did you observe? And what does that indicate about your signal?

_Note that if you did not get a low enough p-value, do not keep trying again and again to meet that threshold - that is called "p-hacking" and is frowned upon by statisticians! Try to make substantial changes to your signal generation code, monitor the mean return to see if it is positive, and then use the statistical test only when you have a clear hypothesis to test._

**A**: 

---

### Optional Extension

A more commonly used significance level is $0.05$ (5% risk, 95% confident). Can you come up with a trading signal that is significant at the $p < 0.05$ level? Go back, and modify the parameters and stock selection logic/thresholds as you need. But make sure you don't change the overall structure of the signal (momentum indicator, long/short portfolio). The functions must retain the same specified input-output interfaces.

**Q**: In case you do attempt to reach the $p = 0.05$ level, or otherwise modify your signal, describe below what formula/procedure/parameters you finally used. What was the resulting mean return, and the exact p-value? Feel free to add more code cells below, as needed.

**A**: 

---

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.